In [ ]:
!pip install gdown

In [18]:
import os
import cv2
import numpy as np
import random
import gdown
import pickle
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [19]:
# with open('../data/external/data.pkl','rb') as f:
#     X_train,y_train,X_test,y_test = pickle.load(f)

In [20]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
# # Define the updated shareable link
# #file_url = 'https://drive.google.com/uc?id=1d76IaKEbiKGAwB1cMGh8FG9g-lI2d9Vf'

# # Define the destination file path where the file will be downloaded
# destination_path = '/content/drive/MyDrive/Capstone/Pickled/'

# # Download the file from the updated shareable link
# #gdown.download(file_url, destination_path, quiet=False)

# # Load the pickled data
# with open(destination_path+'data.pickle', 'rb') as f:
#     X_train, y_train, X_test, y_test = pickle.load(f)

In [22]:
!ls /content/drive/MyDrive/Capstone/Coral_images/

APAL  data_augmented.npz  Pseudodiploria


In [23]:
# Define the path to your image folders
image_folder_path = "/content/drive/MyDrive/Capstone/Coral_images/"

# Initialize lists to store images and labels
images = []
labels = []

# Iterate through each folder in the main directory
for folder_name in os.listdir(image_folder_path):
    folder_path = os.path.join(image_folder_path, folder_name)

    # Check if the item in the main directory is a folder
    if os.path.isdir(folder_path):
        # Iterate through each image file in the subfolder
        for image_filename in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_filename)

            # Load the image using OpenCV
            image = cv2.imread(image_path)

            # Check if the image was loaded successfully
            if image is not None:
                # Resize the image to 1080x1080 if needed
                if image.shape[0] >= 1080 and image.shape[1] >= 1080:
                    image = cv2.resize(image, (1080, 1080))
                else:
                    print(f"Image dimensions are too small for resizing: {image_path}")

                # Append the image to the list of images
                images.append(image)

                # Append the label (folder name) to the list of labels
                labels.append(folder_name)
            else:
                # Handle the case where the image couldn't be loaded
                print(f"Error loading image: {image_path}")

Error loading image: /content/drive/MyDrive/Capstone/Coral_images/Pseudodiploria/.DS_Store


In [24]:
# Convert the lists to NumPy arrays for further analysis
images = np.array(images)
labels = np.array(labels)

# Now, check the shapes
print(f"Images shape: {images.shape}")
print(f"Labels shape: {labels.shape}")

Images shape: (907, 1080, 1080, 3)
Labels shape: (907,)


In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_test shape: {y_test.shape}')

In [ ]:
# Define the data augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    brightness_range=[0.5, 1.5],
    fill_mode='nearest'
)

In [ ]:
# Define the batch size for augmentation
batch_size = 50 # adjust as needed

# Apply data augmentation to the training set
X_train_augmented = []
y_train_augmented = []

for i in range(X_train.shape[0]):
    img = X_train[i]
    label = y_train[i]

    # Expand the dimensions of the image to (1, 1080, 1080, 3)
    img = np.expand_dims(img, axis=0)

    for j in range(2):
        x_augmented = datagen.flow(img, batch_size=1).next()
        X_train_augmented.append(x_augmented[0])  # Extract the augmented image
        y_train_augmented.append(label)
    # Check if it's time to create a new batch
    if len(X_train_augmented) >= batch_size:
        X_train_augmented = np.array(X_train_augmented)
        y_train_augmented = np.array(y_train_augmented)

        #Clearing current batch
        X_train_augmented = []
        y_train_augmented = []

# Convert any remaining data to NumPy arrays and train with the final batch
if X_train_augmented:
    X_train_augmented = np.array(X_train_augmented)
    y_train_augmented = np.array(y_train_augmented)


In [ ]:
# # Convert the augmented data to NumPy arrays
# X_train_augmented = np.array(X_train_augmented)
# y_train_augmented = np.array(y_train_augmented)

: 

In [17]:
print(f'X_train_augmented shape: {X_train_augmented.shape}')
print(f'y_train_augmented shape: {y_train_augmented.shape}')

AttributeError: ignored

In [13]:
# Apply data augmentation to the test set
X_test_augmented = []
y_test_augmented = []

for i in range(X_test.shape[0]):
    img = X_test[i]
    label = y_test[i]

    # Expand the dimensions of the image to (1, 1080, 1080, 3)
    img = np.expand_dims(img, axis=0)

    for j in range(2):
        x_augmented = datagen.flow(img, batch_size=1).next()
        X_test_augmented.append(x_augmented[0])  # Extract the augmented image
        y_test_augmented.append(label)
    if len(X_test_augmented) >= batch_size:
        X_test_augmented = np.array(X_test_augmented)
        y_test_augmented = np.array(y_test_augmented)

        X_test_augmented = []
        y_test_augmented = []

if X_test_augmented:
    X_test_augmented = np.array(X_test_augmented)
    y_test_augmented = np.array(y_test_augmented)

In [14]:
print(f'X_test_augmented shape: {X_test_augmented.shape}')
print(f'y_test_augmented shape: {y_test_augmented.shape}')

X_test_augmented shape: (12, 1080, 1080, 3)
y_test_augmented shape: (12,)


In [ ]:
# print(f'X_train_augmented shape: {X_train_augmented.shape}')
# print(f'y_train_augmented shape: {y_train_augmented.shape}')
# print(f'X_test_augmented shape: {X_test_augmented.shape}')
# print(f'y_test_augmented shape: {y_test_augmented.shape}')

In [15]:
# Save the augmented data to files
np.savez_compressed('/content/drive/MyDrive/Capstone/Coral_images/data_augmented.npz',
                    X_train=X_train_augmented,
                    y_train=y_train_augmented,
                    X_test=X_test_augmented,
                    y_test=y_test_augmented)